In [ ]:
import pandas as pd
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import string

In [ ]:
sds_data = pd.read_csv('../data/sds_ds_text.csv')

In [ ]:
sds_data

In [ ]:
sds_ds_processed = pd.DataFrame(sds_data.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

In [ ]:
sds_ds_processed

In [ ]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [ ]:
sds_ds_processed['episode_split_text'] = sds_ds_processed['episode_split_text'].apply(processed_text)

In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
docs = []
episodes_text = sds_ds_processed['episode_split_text']
for text in episodes_text:
    tokens = simple_preprocess(text)
    docs.append(tokens)

In [ ]:
docs

In [ ]:
from gensim.models import Phrases

In [ ]:
bigram_finder = Phrases(
    sentences = docs,
    min_count = 25
)

In [ ]:
i = 10
bigram_finder[docs[i]]

In [ ]:
trigram_finder = Phrases(
    sentences = bigram_finder[docs],
    min_count = 25
)

In [ ]:
trigram_finder

In [ ]:
i = 10
trigram_finder[bigram_finder[docs[i]]]

In [ ]:
docs = list(trigram_finder[bigram_finder[docs]])

In [ ]:
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

In [ ]:
dictionary.token2id

In [ ]:
dictionary.doc2bow(docs[2])

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
corpus

### Latent Dirichlet Allocation

In [ ]:
from gensim.models import LdaModel

In [ ]:
num_topics = 8            # The number of topics to be extracted
passes = 20               # The number of times to pass through the entire corpus
chunksize = 2000          # The number of documents to be used in a training chunk 
iterations = 400          # The maximum number of iterations through the corpus when inferring the topic distribution

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token     # We need to give the model the id2token dictionary

model = LdaModel(
    corpus = corpus,
    id2word = id2word,
    num_topics = num_topics,
    passes = passes,
    chunksize = chunksize,
    iterations = iterations,
    alpha='auto',         # Learn an asymmetric prior for document-topic distribution from the corpus
    eta='auto',           # Learn an asymmetric prior for topic-word distribution from the corpus
    eval_every = None,    # Speeds up training
    random_state = 321
)

In [ ]:
vis = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, 'lda.html')

### BERTopic

In [15]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string
from umap import UMAP
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance

In [16]:
sds_data = pd.read_csv('../data/sds_ds_text.csv')

<IPython.core.display.Javascript object>

In [17]:
sds_ds_processed = pd.DataFrame(sds_data.groupby(['episode_number', 'episode_name', 'length_episode', 'class', 'guest_name', 'host_episode', 'episode_year', 'episode_date'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [18]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [19]:
sds_ds_processed['episode_split_text'] = sds_ds_processed['episode_split_text'].apply(processed_text)

In [14]:
sds_ds_processed.to_csv('../data/sds_ds_processed.csv', index = False)

### Refer to https://maartengr.github.io/BERTopic/index.html
https://maartengr.github.io/BERTopic/algorithm/algorithm.html#code-overview

In [20]:

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = MaximalMarginalRelevance(diversity=0.7)

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,          # Extract embeddings
  umap_model=umap_model,                    # Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Tokenize topics
  ctfidf_model=ctfidf_model,                # Extract topic words
  representation_model=representation_model # Fine-tune topic represenations
  verbose=True)

In [31]:
docs = sds_ds_processed['episode_split_text'].to_list()
dates = sds_ds_processed['episode_date'].apply(lambda x: pd.Timestamp(x))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
dates

0     2016-09-10
1     2016-09-14
2     2016-09-25
3     2016-10-02
4     2016-10-09
         ...    
446   2023-05-16
447   2023-05-19
448   2023-05-23
449   2023-05-26
450   2023-05-30
Name: episode_date, Length: 451, dtype: datetime64[ns]

In [32]:
topics, probs = topic_model.fit_transform(docs)
freq = topic_model.get_topic_info()

In [33]:
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,222,-1_data_people_really_youre,"[data, people, really, youre, learning, theres...",[this is episode number with chief scientist ...
1,0,57,0_data_really_people_youre,"[data, really, people, youre, science, say, le...",[this is episode number with senior data scie...
2,1,54,1_yeah_learning_theres_kind,"[yeah, learning, theres, kind, things, time, g...",[this is episode number with noah gift founde...
3,2,44,2_data_really_science_going,"[data, really, science, going, time, thats, di...",[this is episode number with aspiring data sc...
4,3,33,3_data_people_science_right,"[data, people, science, right, youre, know, lo...",[this is episode number with data scientist a...
5,4,23,4_data_yeah_intelligence_artificial,"[data, yeah, intelligence, artificial, blockch...",[this is episode number with director of data...
6,5,18,5_model_gpt_llama_billion,"[model, gpt, llama, billion, chinchilla, fivem...",[this is fiveminute friday on open source chat...


In [34]:
fig = topic_model.visualize_topics();
fig

In [36]:
topic_model.topics_over_time(docs, dates, nr_bins=20)

,Topic,Words,Frequency,Timestamp
0,-1,"data, going, really, people, learning",6,2016-09-07 13:07:40.800
1,0,"data, really, like, tableau, people",3,2016-09-07 13:07:40.800
2,2,"data, science, really, going, time",8,2016-09-07 13:07:40.800
3,4,"data, damian, really, going, healthcare",1,2016-09-07 13:07:40.800
4,-1,"data, like, people, think, going",10,2017-01-10 15:36:00.000
...,...,...,...,...
84,-1,"yeah, learning, youre, things, dont",10,2023-01-27 08:24:00.000
85,0,"know, yeah, data, going, make",6,2023-01-27 08:24:00.000
86,1,"data, yeah, theres, youre, people",9,2023-01-27 08:24:00.000
87,4,"ai, thats, lionfish, things, use",1,2023-01-27 08:24:00.000


In [42]:
topics_over_time =  topic_model.topics_over_time(docs=docs, timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, nr_bins=20)

In [43]:
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topics_over_time

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
topic_model.save("prison_model_mini", save_embedding_model=False)
     

In [ ]:
# reduce outliers using best term frequency score
topics = topic_model.topics_
new_topics = topic_model.reduce_outliers(docs, topics, strategy="c-tf-idf")

documents = pd.DataFrame({"Document": docs, "Topic": new_topics})
topic_model._update_topic_size(documents)
topic_model.get_topic_info()

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")